# Handrafted Queries

In [7]:
# THIS SOMEHOW CRASHES THE ENTIRE SCRIPT ... ?!

# q_hc_coauthor = """
# INSERT { 
#     ?authorA <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor> ?authorB 
# } WHERE {
#     ?somepaper <http://purl.org/dc/terms/creator> ?authorA .
#     ?somepaper <http://purl.org/dc/terms/creator> ?authorB .
#     filter not exists {?authorA <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor> ?authorB }.
# }
# """

q_hc_memberOf = """ 
INSERT { 
    ?authorB <http://www.w3.org/ns/org#memberOf> ?someAffiliation .
} WHERE {
    ?authorA <http://www.w3.org/ns/org#memberOf> ?someAffiliation .
    ?authorA <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor> ?authorB .
    ?authorB <http://www.w3.org/ns/org#memberOf> ?someAffiliation .
    filter not exists {?authorB <http://www.w3.org/ns/org#memberOf> ?someAffiliation . } .
}
"""

q_hc_conference = """
INSERT { 
    ?paperB <https://makg.org/property/appearsInConferenceSeries> ?someConference .
} WHERE {
    ?paperA <http://purl.org/spar/cito/cites> ?paperB .
    ?paperA <http://purl.org/dc/terms/creator> ?authorA .
    ?paperB <http://purl.org/dc/terms/creator> ?authorB .
    ?authorA <http://www.w3.org/ns/org#memberOf> ?someAffiliation .
    ?authorB <http://www.w3.org/ns/org#memberOf> ?someAffiliation .
    ?paperA <https://makg.org/property/appearsInConferenceSeries> ?someConference .
    ?paperB <https://makg.org/property/appearsInConferenceSeries> ?someConference . 
    filter not exists {?paperB <https://makg.org/property/appearsInConferenceSeries> ?someConference . }.
}
"""

# Mined Rules

In [8]:
q_m_type = """
INSERT {
    ?paperA a ?typeB .
} WHERE {
    ?paperA <http://purl.org/spar/cito/cites> ?paperH .
    ?paperH a ?typeB .
    FILTER NOT EXISTS{?paperA a ?typeB} .
}
"""

q_m_coauthor = """ 
INSERT {
    ?authorA <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor> ?authorB .
} WHERE {
    ?authorB <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor> ?authorA .
    FILTER NOT EXISTS{?authorA <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor> ?authorB} .
}
"""

q_m_conference = """
INSERT {
    ?paperA <https://makg.org/property/appearsInConferenceSeries> ?seriesB .
} WHERE {
    ?paperA <http://purl.org/spar/cito/cites> ?paperH .
    ?paperH <https://makg.org/property/appearsInConferenceSeries> ?seriesB .
    FILTER NOT EXISTS{?paperA <https://makg.org/property/appearsInConferenceSeries> ?seriesB} .
}
"""

q_m_memberOf = """
INSERT {
    ?authorA <http://www.w3.org/ns/org#memberOf> ?affiliationB .
} WHERE {
    ?authorG <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor> ?authorA .
    ?authorG <http://www.w3.org/ns/org#memberOf> ?affiliationB .
    FILTER NOT EXISTS{?authorA <http://www.w3.org/ns/org#memberOf> ?affiliationB} .
}
"""

# Apply Queries to Knowledge Graph

In [9]:
from rdflib import Graph
import copy

In [12]:
g = Graph()
g.parse('train.nt', format='nt')
gnew = copy.deepcopy(g)

In [13]:
print(f"Original Graph Triple Count: {len(g)}\n")
len_counter = len(g)

for q in [q_hc_memberOf, q_hc_conference, q_m_type, q_m_coauthor, q_m_conference, q_hc_memberOf]:
    gnew.update(q)
    print(f'Inserted Triples: {len(gnew) - len_counter}')
    len_counter = len(gnew)
    print(f'New Graph Triple Count: {len(gnew)}')
    print()


Original Graph Triple Count: 89253

Inserted Triples: 0
New Graph Triple Count: 89253

Inserted Triples: 0
New Graph Triple Count: 89253

Inserted Triples: 0
New Graph Triple Count: 89253

Inserted Triples: 270
New Graph Triple Count: 89523

Inserted Triples: 2904
New Graph Triple Count: 92427

Inserted Triples: 0
New Graph Triple Count: 92427



In [6]:
gnew.serialize('train_full.nt', format='nt')

c:\Users\johan\AppData\Local\Programs\Python\Python39\lib\site-packages\rdflib\plugins\serializers\nt.py:35: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


<Graph identifier=N52dca8da80bb464ea2769b637613ccab (<class 'rdflib.graph.Graph'>)>

In [17]:
with open('train_full.nt', 'r', encoding='utf-8', errors='ignore') as file:
    lines = file.readlines()
    
with open('train_full.nt', 'w', encoding='utf-8') as file:
    for line in lines:
        file.write(line.replace(' ', '\t'))

In [30]:
!java -jar amie.jar train_full.nt > task3_result.txt

Using the default schema relations


In [38]:
with open('./task3_result.txt', 'r', encoding='utf-8', errors='ignore') as file:
    lines = file.readlines()

with open('./task3_result.csv', 'w', encoding='utf-8') as file:
    for line in lines:
        line = line.replace('\t', ',')
        file.write(line)


In [39]:
import pandas as pd

DISPLAY_ALL_TEXT = False
pd.set_option("display.max_colwidth", 0 if DISPLAY_ALL_TEXT else 50)


In [41]:
rules_df = pd.read_csv('task3_result.csv')
conf_sort_df = rules_df.sort_values('Std Confidence', ascending=False)
conf_sort_df

,Rule,Head Coverage,Std Confidence,PCA Confidence,Positive Examples,Body size,PCA Body size,Functional variable
9,?g <http://purl.org/spar/cito/cites> ?a ?g ...,0.149126,1.000000,1.000000,2440,2440,2440,-1
8,?a <http://purl.org/spar/cito/cites> ?h ?h ...,0.171678,1.000000,1.000000,2809,2809,2809,-1
16,?g <http://purl.org/dc/terms/creator> ?a ?g...,0.981375,1.000000,1.000000,32511,32511,32511,-1
13,?g <http://lsdis.cs.uga.edu/projects/semdis/o...,0.495844,1.000000,1.000000,8113,8113,8113,-1
12,?a <http://lsdis.cs.uga.edu/projects/semdis/o...,0.495844,1.000000,1.000000,8113,8113,8113,-1
0,?b <http://lsdis.cs.uga.edu/projects/semdis/o...,0.999849,0.999849,0.999849,33123,33128,33128,-1
1,?a <http://purl.org/spar/cito/cites> ?h ?h ...,0.634733,0.683076,0.684921,4565,6683,6665,-1
2,?g <http://purl.org/spar/cito/cites> ?a ?g ...,0.437987,0.404572,0.424987,3150,7786,7412,-1
11,?g <http://lsdis.cs.uga.edu/projects/semdis/o...,0.908600,0.370879,0.479645,5726,15439,11938,-1
4,?a <http://lsdis.cs.uga.edu/projects/semdis/o...,0.908600,0.370807,0.479564,5726,15442,11940,-1


In [42]:
for rule in conf_sort_df['Rule']:
    print(rule)


?g  <http://purl.org/spar/cito/cites>  ?a  ?g  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?b   => ?a  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?b
?a  <http://purl.org/spar/cito/cites>  ?h  ?h  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?b   => ?a  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?b
?g  <http://purl.org/dc/terms/creator>  ?a  ?g  <http://purl.org/dc/terms/creator>  ?b   => ?a  <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor>  ?b
?g  <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor>  ?a  ?g  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?b   => ?a  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?b
?a  <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor>  ?h  ?h  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?b   => ?a  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?b
?b  <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor>  ?a   => ?a  <http://lsdis.cs.uga.edu/projects/semdis/opus#coauthor>  ?b
?a  <http:/